In [21]:
import pandas as pd

games_df = pd.read_csv('../data/src/games.csv')
boxscore_df = pd.read_csv('../data/src/boxscore.csv')
boxscore_df.head()

,game_id,team_name,player_name,MP,FG,FGA,3P,3PA,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,is_starter
0,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Corliss Williamson,37:20,7.0,11.0,0.0,0.0,0.0,0.0,...,3.0,4.0,4.0,1.0,1.0,4.0,5.0,14.0,-2.0,1
1,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Mitch Richmond,32:00,6.0,12.0,1.0,4.0,1.0,1.0,...,5.0,5.0,3.0,1.0,2.0,3.0,1.0,14.0,-12.0,1
2,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Olden Polynice,31:34,0.0,4.0,0.0,0.0,1.0,4.0,...,5.0,7.0,3.0,1.0,0.0,4.0,3.0,1.0,-12.0,1
3,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Mahmoud Abdul-Rauf,29:27,7.0,13.0,1.0,2.0,2.0,2.0,...,2.0,2.0,5.0,1.0,1.0,2.0,2.0,17.0,-7.0,1
4,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Brian Grant,25:13,3.0,11.0,0.0,0.0,2.0,2.0,...,5.0,6.0,0.0,0.0,2.0,1.0,2.0,8.0,-7.0,1


In [22]:
mask = boxscore_df.isin(['Player Suspended', 'Did Not Play', 'Did Not Dress', 'Not With Team']).any(axis=1)
boxscore_df = boxscore_df[~mask]
for col in ['FG','FGA','3P','3PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','+/-']:
    boxscore_df[col] = pd.to_numeric(boxscore_df[col])

In [30]:
grouped = boxscore_df.groupby(['game_id', 'team_name'])

# First, aggregate the stats in your main dataframe
aggregated_df = grouped[['FG','FGA','3P','3PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','+/-']].sum().reset_index()

# Now, merge with the games_df to get the datetime and is_regular values
aggregated_df = pd.merge(aggregated_df, games_df[['game_id', 'datetime', 'is_regular']], on='game_id', how='left')

aggregated_df['win'] = aggregated_df.groupby('game_id')['PTS'].transform(lambda x: x == x.max())
aggregated_df.head()


,game_id,team_name,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,...,AST,STL,BLK,TOV,PF,PTS,+/-,datetime,is_regular,win
0,000079d5-37c3-4893-9f2c-d0776e80640c,New York Knicks,36.0,90.0,4.0,21.0,14.0,20.0,15.0,33.0,...,18.0,8.0,5.0,9.0,24.0,90.0,-20.0,2014-02-05,1,False
1,000079d5-37c3-4893-9f2c-d0776e80640c,Portland Trail Blazers,30.0,79.0,8.0,21.0,26.0,33.0,12.0,37.0,...,17.0,5.0,5.0,14.0,16.0,94.0,20.0,2014-02-05,1,True
2,00034685-1fcc-4465-ab33-473de0d6ba31,Houston Rockets,42.0,86.0,21.0,49.0,8.0,9.0,13.0,37.0,...,24.0,10.0,5.0,17.0,16.0,113.0,115.0,2019-03-24,1,True
3,00034685-1fcc-4465-ab33-473de0d6ba31,New Orleans Pelicans,38.0,92.0,7.0,28.0,7.0,10.0,15.0,29.0,...,26.0,9.0,3.0,15.0,9.0,90.0,-115.0,2019-03-24,1,False
4,000702bc-3181-48fa-88ce-6e0fabc44917,Denver Nuggets,43.0,94.0,3.0,13.0,21.0,24.0,14.0,30.0,...,27.0,8.0,1.0,11.0,17.0,110.0,50.0,2004-04-10,1,True


In [33]:
aggregated_df = aggregated_df.sort_values(by=['datetime'])
aggregated_df.to_csv('../data/src/boxscore_aggregates.csv', index=False)